<div style="display: flex; align-items: left;">
    <a href="https://sites.google.com/corp/google.com/genai-solutions/home?authuser=0">
        <img src="https://storage.googleapis.com/miscfilespublic/Linkedin%20Banner%20%E2%80%93%202.png" style="margin-right">
    </a>
</div>


<h1 align="center">Open Data QnA - Chat with your SQL Database</h1> 

---

This notebook first walks through the Vector Store Setup needed for running the Open Data QnA application. 

Currently supported Source DBs are: 
- PostgreSQL on Google Cloud SQL 
- BigQuery

Furthermore, the following vector stores are supported 
- pgvector on PostgreSQL 
- BigQuery vector


The setup part covers the following steps: 
> 1. Configuration: Intial GCP project, IAM permissions, Environment  and Databases setup including logging on Bigquery for analytics

> 2. Creation of Table, Column and Known Good Query Embeddings in the Vector Store  for Retreival Augmented Generation(RAG)


Afterwards, you will be able to run the Open Data QnA Pipeline to generate SQL queries and answer questions over your data source. 

The pipeline run covers the following steps: 

> 1. Take user question and generate sql in the dialect corresponding to data source

> 2. Execute the sql query and retreive the data

> 3. Generate natural language respose and charts to display

> 4. Clean Up resources



### 📒 Using this interactive notebook

If you have not used this IDE with jupyter notebooks it will ask for installing Python + Jupyter extensions. Please go ahead install them

Click the **run** icons ▶️  of each cell within this notebook.

> 💡 Alternatively, you can run the currently selected cell with `Ctrl + Enter` (or `⌘ + Enter` on a Mac).

> ⚠️ **To avoid any errors**, wait for each section to finish in their order before clicking the next “run” icon.

This sample must be connected to a **Google Cloud project**, but nothing else is needed other than your Google Cloud project.

You can use an existing project. Alternatively, you can create a new Cloud project [with cloud credits for free.](https://cloud.google.com/free/docs/gcp-free-tier)

# 🚧 **0. Prerequisites**

Make sure that Google Cloud CLI is installed before moving to the next cell! You can refer to the link below for guidance

Installation Guide: https://cloud.google.com/sdk/docs/install

### Set Python Module Path to Root

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)



In [2]:
%load_ext autoreload
%autoreload 2
%aimport

import logging
format_string = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='mylog.log', mode='a')
formatter = logging.Formatter(format_string)
fhandler.setFormatter(formatter)
#logger.addHandler(fhandler)
logging.basicConfig(format=format_string,
                     level=logging.INFO, stream=sys.stdout)
logger.setLevel(logging.INFO)

Modules to reload:
all-except-skipped

Modules to skip:



### 🔗 **Connect Your Google Cloud Project**
Time to connect your Google Cloud Project to this notebook. 

In [3]:
#@markdown Please fill in the value below with your GCP project ID and then run the cell.
PROJECT_ID = "uk-bh-experiments-argolis"

# Quick input validations.
assert PROJECT_ID, "⚠️ Please provide your Google Cloud Project ID"

# Configure gcloud.
!gcloud config set project {PROJECT_ID}
print(f'Project has been set to {PROJECT_ID}')

os.environ['GOOGLE_CLOUD_QUOTA_PROJECT']=PROJECT_ID
os.environ['GOOGLE_CLOUD_PROJECT']=PROJECT_ID

Updated property [core/project].
Project has been set to uk-bh-experiments-argolis


### 🤔 **Choose Data Source and Vector Store**

Fill out the parameters and configuration settings below. 
These are the parameters for connecting to the source databases and setting configurations for the vector store tables to be created. 
Additionally, you can specify whether you have and want to use known-good-queries for the pipeline run and whether you want to enable logging.

**Known good queries:** if you have known working user question <-> SQL query pairs, you can put them into the file `scripts/known_good_sql.csv`. This will be used as a caching layer and for in-context learning: If an exact match of the user question is found in the vector store, the pipeline will skip SQL Generation and output the cached SQL query. If the similarity score is between 90-100%, the known good queries will be used as few-shot examples by the SQL Generator Agent. 

**Logging:** you can enable logging. If enabled, a dataset is created in Big Query in your project, which will store the logging table and save information from the pipeline run in the logging table. This is especially helpful for debugging.

In [4]:
#[CONFIG]
embedding_model = 'vertex' # Options: 'vertex' or 'vertex-lang'
vector_embedding_model = "text-embedding-004"
description_model = 'gemini-1.5-pro-001' # 'gemini-1.5-pro-001', 'gemini-1.5-pro', 'text-bison-32k'
data_source = 'bigquery' #  Options: 'bigquery' and 'cloudsql-pg' 
vector_store = 'bigquery-vector' # Options: 'bigquery-vector', 'cloudsql-pgvector'
logging = True # True or False 
kgq_examples = True # True or False 

#[GCP]
project_id = PROJECT_ID

#[PGCLOUDSQL]
# If you want to use PG as source, fill out the values below
pg_region = ''
pg_instance = ''
pg_database = ''
pg_user = ''
pg_password = ''
pg_schema = ''

#[BIGQUERY]
# If you want to use BQ as source, fill out the values below
bq_dataset_region = 'us-east11'
bq_dataset_name = 'breedr'

# Name for the BQ dataset created for bigquery-vector and/or logging
bq_opendataqna_dataset_name = 'opendataqna'
bq_log_table_name = 'audit_log_table' 
bq_table_list = None # ["animal_breeds", "activity_types", "alert_type", "activity_types"]  # either None or a list of table names in format ['reviews', 'ratings']

#Decode Region and Userdatabase based on source
if data_source == 'bigquery': dataset_region = bq_dataset_region; user_database=bq_dataset_name 
elif data_source == 'cloudsql-pg': dataset_region = pg_region; user_database=pg_schema

Quick input verifications below:

In [5]:

# Input verification - Source
assert data_source in {'bigquery', 'cloudsql-pg'}, "⚠️ Invalid DATA_SOURCE. Must be 'bigquery' or 'cloudsql-pg'"

# Input verification - Vector Store
assert vector_store in {'bigquery-vector', 'cloudsql-pgvector'}, "⚠️ Invalid VECTOR_STORE. Must be 'bigquery-vector' or 'cloudsql-pgvector'"

if logging: 
    assert bq_log_table_name, "⚠️ Please provide a name for your log table if you want to use logging"

if data_source == 'bigquery':
    assert bq_dataset_region, "⚠️ Please provide the Data Set Region"
    assert bq_dataset_name, "⚠️ Please provide the name of the dataset on Bigquery"

elif data_source == 'cloudsql-pg':
    assert pg_region, "⚠️ Please provide Region of the Cloud SQL Instance"
    assert pg_instance, "⚠️ Please provide the name of the Cloud SQL Instance"
    assert pg_database, "⚠️ Please provide the name of the PostgreSQL Database on the Cloud SQL Instance"
    assert pg_user, "⚠️ Please provide a username for the Cloud SQL Instance"
    assert pg_password, "⚠️ Please provide the Password for the PG_USER"


### 💾 **Save Configuration to File** 
Save the configurations set in this notebook to  `config.ini`. The parameters from this file are used in notebooks and in various modeules in the repo

In [6]:
from scripts import save_config

save_config(embedding_model, vector_embedding_model, description_model, data_source, vector_store, logging, kgq_examples, PROJECT_ID,
            pg_region, pg_instance, pg_database, pg_user, pg_password, pg_schema, 
            bq_dataset_region, bq_dataset_name, 
            bq_opendataqna_dataset_name, bq_log_table_name, bq_table_list)

current dir:  /home/jupyter/uk-bh-experiments/bhfork-opendataqna/applied-ai-engineering-samples/notebooks
All configuration paramaters saved to file!


##  **1.2. Create Embeddings in Vector Store for RAG** 

### 🖋️ **Create Table and Column Embeddings**

In this step, table and column metadata is retreived from the data source and embeddings are generated for both

In [7]:
from env_setup import get_embeddings

# Generate embeddings for tables and columns
table_schema_embeddings, col_schema_embeddings = get_embeddings()  



current dir:  /home/jupyter/uk-bh-experiments/bhfork-opendataqna/applied-ai-engineering-samples/notebooks
root_dir set to: /home/jupyter/uk-bh-experiments/bhfork-opendataqna/applied-ai-engineering-samples
2024-06-26 11:03:02,335 - agents.core - INFO - LLM Model is gemini-1.5-pro-001 
Generating embeddings from source db schemas
2024-06-26 11:03:04,815 - agents.DescriptionAgent - INFO - Generate description for table uk-bh-experiments-argolis.breedr.business_units
2024-06-26 11:03:06,860 - agents.DescriptionAgent - INFO - Generate description for table uk-bh-experiments-argolis.breedr.cts_dairy_breeds
2024-06-26 11:03:08,141 - agents.DescriptionAgent - INFO - Generate description for table uk-bh-experiments-argolis.breedr.unit_of_measurement_business_unit
2024-06-26 11:03:09,512 - agents.DescriptionAgent - INFO - Generate description for table uk-bh-experiments-argolis.breedr.supply_chain
2024-06-26 11:03:10,722 - agents.DescriptionAgent - INFO - Generate description for table uk-bh-exp

In [8]:
table_schema_embeddings.describe()


,table_schema,table_name,content,embedding
count,45,45,45,45
unique,1,45,45,45
top,breedr,business_units,\n Full Table Name : uk-bh-experime...,"[0.022505661472678185, 0.028678087517619133, -..."
freq,45,1,1,1


In [9]:
col_schema_embeddings.describe()

,table_schema,table_name,column_name,content,embedding
count,631,631,631,631,631
unique,1,45,374,631,631
top,breedr,animals,id,\n Column Name: breedr.account_user...,"[0.0012524769408628345, 0.07620758563280106, -..."
freq,631,63,41,1,1


### 💾 **Save the Table and Column Embeddings in the Vector Store**
The table and column embeddings created in the above step are save to the Vector Store chosen

In [10]:
from env_setup import store_embeddings


# Store table/column embeddings (asynchronous)
await(store_embeddings(table_schema_embeddings, col_schema_embeddings)) 


Storing embeddings back to the vector store.
2024-06-26 11:18:15,144 - embeddings.store_embeddings - INFO - Clean up table_details_embeddings: query_string=" DELETE FROM `uk-bh-experiments-argolis.opendataqna.table_details_embeddings` WHERE table_schema = 'breedr' AND table_name IN( 'business_units','cts_dairy_breeds','unit_of_measurement_business_unit','supply_chain','medicine_vmd_data','animals_identifierassignment','animal_types','medicine_lineitem','predictions_when_weighed','countries','animals_tag_change','lots','animals_identifierpreferences','kill_information','animals','animals_tag_value','currencies','medicine_stockitem_species','medicine_withdrawals','crush_app_sessions','activity_types','animals2','groups','pasture_types','business_unit_users','animals_tag','organizations_organisation','medicine_itemtransaction','medicine_stockitem','medicines','offers','supply_chain_member','group_reports','activities','animals_animal_breeds','reports_business_types','business_types','anim

### 🗄️ **Load Known Good SQL into Vector Store**
Known Good Queries are used to create query cache for Few shot examples. Creating a query cache is highly recommended for best outcomes! 

The following cell will load the Natural Language Question and Known Good SQL pairs into our Vector Store. There pairs are loaded from `known_good_sql.csv` file inside scripts folder. If you have your own Question-SQL examples, curate them in .csv file before running the cell below. 

If no Known Good Queries are available at this time to create query cache, you can use [3_LoadKnownGoodSQL.ipynb](3_LoadKnownGoodSQL.ipynb) to load them later. 



#### Format of the Known Good SQL File (known_good_sql.csv)

prompt | sql | database_name [3 columns]

prompt ==> User Question 

sql ==> SQL for the user question (Note that the sql should enclosed in quotes and only in single line. Please remove the line  break)

database_name ==>This name should exactly  match the SCHEMA   NAME for Postgres Source or BQ_DATASET_NAME

In [11]:
from env_setup import create_kgq_sql_table, store_kgq_sql_embeddings

# Create table for known good queries (if enabled)
await(create_kgq_sql_table()) 

# Store known good query embeddings (if enabled)
await(store_kgq_sql_embeddings())  


Creating kgq table in vector store.
Reading contents of known_good_sql.csv
Known Good SQL Found at Path :: /home/jupyter/uk-bh-experiments/bhfork-opendataqna/applied-ai-engineering-samples/scripts/known_good_sql.csv
Storing kgq embeddings in vector store table.
kgq embeddings stored.


### 🥁 If all the above steps are executed successfully, the following should be set up:

* GCP project and all the required IAM permissions

* Environment to run the solution

* Data source and Vector store for the solution

__________________________________________________________________________________________________________________